In [3]:
!pip install --q crewai
!pip install --q crewai-tools

In [ ]:
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
load_dotenv()

C:\Users\005255\AppData\Roaming\Python\Python312\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
C:\Users\005255\AppData\Roaming\Python\Python312\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
C:\Users\005255\AppData\Roaming\Python\Python312\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` valida

In [ ]:
gpt4o = "gpt-4o"
gpt4omini = "gpt-4o-mini"
ferramenta_pesquisa = SerperDevTool()

In [5]:
#Função callback do pesquisador para acompanhamento em tempo real

def registrar_processo ( step_info ) :
  print ( step_info )


#Dicionários de configurações
pesquisadorConfig = {
  'role':'Pesquisador',
  'backstory':f'Você, como pesquisador de conte údo na web ... ',
  'goal':'Coletar e analisar ... ',
  'verbose':False,
  'llm':gpt4omini,
  'max_inter':15,
  'max_rpm':30,
  'memory':True,
  'function_calling_llm':gpt4o,
  'max_execution_time':120,
  # 'allow_code_execution':True, - Tive que desativar pois o colab não fornece suporte ao docker que é necessário para essa propriedade
  'step_callback':registrar_processo,
  'tools':[ ferramenta_pesquisa ]
}

coordenadorConfig = {
  'role':'Coordenador',
  'backstory':f'Você é um coordenador experiente ...',
  'goal':'Supervisionar e integrar ...',
  'verbose':False,
  'llm':gpt4omini,
  'allow_delegation':True,
  'memory':True,
  'max_retry_limit':3
}

escritorConfig = {
  'role':'Escritor',
  'backstory':f'Você é um escritor talentoso ...',
  'goal':'Criar conteúdo escrito ... ',
  'verbose':False,
  'llm':gpt4o
}


In [6]:
pesquisador = Agent (config = pesquisadorConfig)
coordenador = Agent (config = coordenadorConfig)
escritor = Agent (config = escritorConfig)

In [7]:
def notificar_escritor(result):
  print(f"Tarefa de pesquisa concluída. ResultadoS: {result}")

In [8]:
pesquisa_tendencias_config = {
    'agent': pesquisador,
    'description': 'Pesquise as tendências do mercado de Inteligência Artificial no Brasil a partir de janeiro de 2025. Utilize fontes como blogs, artigos científicos, sites de noticias e redes sociais para compilar um relatório detalhado que descreva as principais inovações e discussões na área.',
    'expected_output': 'Um relatório detalhado de 10 páginas que destaque as tendências principais de IA no Brasil, incluíndo pelo menos 3 inovações emergentes e 5 fontes referências de fontes variadas.',
    'tools': [ ferramenta_pesquisa ],
    'async_execution' : True,
    'callback' : notificar_escritor
}

coordenador_pesquisa_config = {
    'agent' : coordenador,
    'description' : 'Supervisione o andamento do projeto , integrando os resultados das tarefas realizadas pelos diferentes agentes. Certifique-se de que todas as partes do projeto estejam alinhadas e funcionando de forma coesa. Relate qualquer problema encontrado e proponha soluções para melhorar a eficiência do projeto .',
    'expected_output' : 'Um relatório detalhado sobre o progresso do projeto, incluindo problemas identificados e soluções propostas'
}

criar_conteudo_config = {
    'agent' : escritor,
    'description' : 'Crie conteúdo escrito baseado nos dados e análises fornecidas pelo Coordenador e outros agentes. O conteúdo deve ser claro, envolvendo e informativo, abordando os principais pontos e conclusões do projeto',
    'expected_output':'Um artigo de 1500 palavras sobre os resuldados e conclusões do projeto, pronto para publicação, colocando pelo menos 2 fontes usadas na pesquisa',
    'tools': [ ferramenta_pesquisa ],
}

In [9]:
pesquisa_tendencias_task = Task(config = pesquisa_tendencias_config)
coordenador_pesquisa_task = Task(config = coordenador_pesquisa_config)
criar_conteudo_task = Task(config = criar_conteudo_config)

In [10]:
crew = Crew(
    agents=[pesquisador, escritor],
    tasks=[coordenador_pesquisa_task, pesquisa_tendencias_task, criar_conteudo_task],
    process = Process.hierarchical,
    manager_agent=coordenador,
    manager_llm = gpt4omini,
    memory = True,
    verbose = True,
    function_calling_llm = gpt4omini,
    cache = True,
)

In [12]:
result = crew.kickoff()
print(result)

# Agent: Coordenador
## Task: Supervisione o andamento do projeto , integrando os resultados das tarefas realizadas pelos diferentes agentes. Certifique-se de que todas as partes do projeto estejam alinhadas e funcionando de forma coesa. Relate qualquer problema encontrado e proponha soluções para melhorar a eficiência do projeto .


# Agent: Coordenador
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": {\"description\": \"Supervisione o andamento do projeto, integrando os resultados das tarefas realizadas pelos diferentes agentes. Certifique-se de que todas as partes do projeto estejam alinhadas e funcionando de forma coesa. Relate qualquer problema encontrado e proponha solu\\u00e7\\u00f5es para melhorar a efici\\u00eancia do projeto.\", \"type\": \"str\"}, \"context\": {\"description\": \"Por favor, compile um relat\\u00f3rio detalhado sobre o progresso do projeto, incluindo os problemas identificados e solu\\u00e7\\u00f5es propostas. Utilize dados que possam quan